In [1]:
%load_ext tensorboard
import tensorflow as tf
import tensorflow_datasets as tfds
from absl import app
from absl import flags
import numpy as np
from spiking_tf.src import spiking_models, plots, file_handling
import datetime

In [2]:
timesteps = 1
max_rate = 2

n_in = 28*28
n_hidden = 800
n_out = 10

batch_size = 128
epochs = 10

output_path = file_handling.get_default_path_str()

In [3]:
def flatten(image, label):
    '''Transform image to the flattened version of itself'''
    flattened = tf.reshape(image, [28*28])
    flattened = tf.expand_dims(flattened, 0)
    return tf.cast(flattened, tf.float32), label

In [4]:
tf.random.set_seed(1234)

(ds_train, ds_test) = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True
)
ds_train = ds_train.map(
    flatten, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(batch_size)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
print(ds_train)

ds_test = ds_test.map(
    flatten, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

<PrefetchDataset shapes: ((None, 1, 784), (None,)), types: (tf.float32, tf.int64)>


In [5]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(800, activation='relu', input_shape=(784,), use_bias=False),
  tf.keras.layers.Dense(10, use_bias=False),
])

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["sparse_categorical_accuracy"],
    run_eagerly=False
)

log_dir = "logs/fit/analog_feedforward" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(
    ds_train, epochs=epochs, validation_data=ds_test, callbacks=[tensorboard_callback]
)

Epoch 1/10


  1/469 [..............................] - ETA: 0s - loss: 177.5513 - sparse_categorical_accuracy: 0.1016WARNING:tensorflow:From /home/viktor/.local/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


  2/469 [..............................] - ETA: 15s - loss: 155.3503 - sparse_categorical_accuracy: 0.1289WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0162s vs `on_train_batch_end` time: 0.0508s). Check your callbacks.


461/469 [============================>.] - ETA: 0s - loss: 5.0067 - sparse_categorical_accuracy: 0.9104WARNING:tensorflow:Model was constructed with shape (None, 784) for input Tensor("dense_input:0", shape=(None, 784), dtype=float32), but it was called on an input with incompatible shape (None, 1, 784).


469/469 [==============================] - 2s 5ms/step - loss: 4.9389 - sparse_categorical_accuracy: 0.9109 - val_loss: 0.7993 - val_sparse_categorical_accuracy: 0.9415
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.4646 - sparse_categorical_accuracy: 0.9589 - val_loss: 0.5392 - val_sparse_categorical_accuracy: 0.9531
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.2562 - sparse_categorical_accuracy: 0.9695 - val_loss: 0.4516 - val_sparse_categorical_accuracy: 0.9608
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1867 - sparse_categorical_accuracy: 0.9748 - val_loss: 0.4566 - val_sparse_categorical_accuracy: 0.9594
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1968 - sparse_categorical_accuracy: 0.9752 - val_loss: 0.5645 - val_sparse_categorical_accuracy: 0.9525
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.2030 - sparse_categorical_accuracy:

In [6]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [19]:
%tensorboard --logdir logs/fit